In [1]:
%pip install --quiet ragas
%pip install --quiet python-dotenv

from dotenv import load_dotenv
load_dotenv()




[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


True

In [3]:

import json
import os
from pathlib import Path
import re
from typing import List, Optional
from pydantic import BaseModel
from langchain.storage import LocalFileStore
from langchain_core.documents import Document

from langchain.storage.exceptions import InvalidKeyException
from langchain_core.stores import BaseStore


from datetime import date
from uuid import UUID
from typing import List

product_images_path = Path("../initial-training-sets/datasets/images/")

psi_file_path = Path("../V1_DATA/psi-llama3.1:8b-instruct-q5_k_m.json")
pid_file_path = Path("../V1_DATA/pid-llava:13b-v1.6-vicuna-q4_0.json")
raw_data_file_path = Path("../V1_DATA/raw_products.json")

assert raw_data_file_path.exists()
assert pid_file_path.exists()
assert psi_file_path.exists()

class RawReview(BaseModel):
    id: str
    review_ref: str
    product_id: str
    review_content: str
    review_title: Optional[str]
    date_written: date
    product_asin: str
    helpful_count: int
    rating_given: int
    review_page_url: str

class RAWProduct(BaseModel):
    id: str
    name: str
    description: str
    product_asin: str
    overall_ratings: float
    total_customers_that_rated: int
    price: float
    currency: str
    category: str
    sub_category: str
    product_page_url: str
    image_url: str
    reviews: List[RawReview]
    
    
class ProductCombinedInformation:
    
    product: RAWProduct
    
    def __init__(self, product: RAWProduct) -> None:
        self.product = product
        
    def get_document(self):
        return Document(page_content=self.info(), id=str(self.product.id), metadata={
            "product_id": self.product.id,
            "product_asin": self.product.product_asin,
            "image_url": self.product.image_url,
            "name": self.product.name
        })
    
    
    def info(self):
        return (
            f"Product ID: {str(self.product.id).strip()} \n"
            f"Product Name: {str(self.product.name).strip()} \n"
            f"Product Description: {str(self.product.description).strip()} \n"
            f"Product Asin: {self.product.product_asin} \n"
            f"Overall Ratings {self.product.overall_ratings} \n"
            f"Total Customers that rated: {self.product.total_customers_that_rated} \n"
            f"Pric: {self.product.currency}{self.product.price} \n"   
        )
        
    def image_path(self):
        return f"{product_images_path}/{self.product.product_asin}.png"
    
    
    psi_file_path = Path("../V1_DATA/psi-llama3.1:8b-instruct-q5_k_m.json")
pid_file_path = Path("../V1_DATA/pid-llava:13b-v1.6-vicuna-q4_0.json")
raw_data_file_path = Path("../V1_DATA/raw_products.json")

assert raw_data_file_path.exists()
assert pid_file_path.exists()
assert psi_file_path.exists()

product_summaries = json.loads(psi_file_path.read_text())
product_image_descriptions = json.loads(pid_file_path.read_text())
raw_products = json.loads(raw_data_file_path.read_bytes())


products_combined_infos = {}
for rp in raw_products:
    if rp["product_asin"] in product_summaries:
        products_combined_infos[rp["product_asin"]] = ProductCombinedInformation(
            product=RAWProduct.model_validate(rp)
        )


# Everything must be in equal
# TODO: Check that same item is in the exact position in all dict
assert (
    len(product_image_descriptions)
    == len(product_summaries)
    == len(products_combined_infos)
)

product_image_descriptions = sorted(
    product_image_descriptions.items(), key=lambda x: x[0]
)
product_summaries = sorted(product_summaries.items(), key=lambda x: x[0])
products_combined_infos = sorted(
    products_combined_infos.items(), key=lambda x: x[0]
)


In [4]:

products_combined_infos = products_combined_infos[0:20]
product_summaries = product_summaries[0:20]
product_image_descriptions = product_image_descriptions[0:20]

In [5]:
## Product Info Documents
product_info_docs = [
    Document(
        page_content=product_info.info(),
        metadata={
            "id": product_info.product.id,
            "name": product_info.product.name,
            "product_asin": product_info.product.product_asin,
        },
    )
    for product_asin, product_info in products_combined_infos
]

len(product_info_docs)

20

In [6]:
## Product Info Summary Documents
summary_docs = [
    Document(
        page_content=summary_text,
        metadata={
            "product_asin": product_asin,
        },
    )
    for product_asin, summary_text in product_summaries
]

len(summary_docs)

20

In [7]:
## Product Image Description
image_descriptions_docs = [
    Document(
        page_content=description_text,
        metadata={
            "product_asin": product_asin,
        },
    )
    for product_asin, description_text in product_image_descriptions
]

len(image_descriptions_docs)

20

In [13]:

import enum
class TextLLM(enum.StrEnum):
    GPT_4O = "gpt-4o"
    LLAMA_3_1 = "llama3.1"
    LLAMA_3_1_INSTRUCT = "llama3.1:8b-instruct-q5_K_M"
    MISTRAL = "mistral"


class VisionLLM(enum.StrEnum):
    GPT_4O = "gpt-4o"
    LLAVA = "llava"
    LLAVA_VICUNA_Q4_0 = "llava:13b-v1.6-vicuna-q4_0".lower()


In [14]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.chat_models import ChatOllama

# documents = load your documents


from langchain_community.embeddings.ollama import OllamaEmbeddings
text_embedding_model_name = "nomic-embed-text"
# for nomic-embed, embed_instruction is `search_document` to embed documents for RAG and `search_query` to embed the question
embeddings = OllamaEmbeddings(
    model=text_embedding_model_name,
    embed_instruction="search_document",
    query_instruction="search_query",
)
generator_llm = ChatOllama(model=TextLLM.MISTRAL)
critic_llm = ChatOllama(model=TextLLM.LLAMA_3_1_INSTRUCT)

# generator with openai models
# generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
# critic_llm = ChatOpenAI(model="gpt-4")
# embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

# Change resulting question type distribution
distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

# use generator.generate_with_llamaindex_docs if you use llama-index as document loader
testset = generator.generate_with_langchain_docs(summary_docs, 10, distributions) 
testset.to_pandas()


Filename and doc_id are the same for all nodes.                 
Generating: 100%|██████████| 10/10 [00:43<00:00,  4.36s/it]


,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What materials are used to make the Men's Tasm...,[Men's Tasman Slip-on Slipper \nMade of 80% Wo...,The Men's Tasman Slip-on Slipper is made of 80...,simple,[{'product_asin': 'B000J1577Q'}],True
1,What occasions are Men's Tom Cats Oxford shoes...,[Men's Tom Cats Oxford shoes are made from hig...,Men's Tom Cats Oxford shoes are suitable for f...,simple,[{'product_asin': 'B0007TLJ6S'}],True
2,What material are Chucks Red M9621C Red All St...,[Chucks Red M9621C Red All Star HI are classic...,Chucks Red M9621C Red All Star HI sneakers are...,simple,[{'product_asin': 'B000072UPN'}],True
3,What is the purpose of the 300NBS rubber outso...,[Kingpin Men's Low-Top Trainers are made from ...,The purpose of the 300NBS rubber outsole in th...,simple,[{'product_asin': 'B000B3NE1I'}],True
4,What material is the case back of the Casio Me...,[Casio Men's MQ24-7B2 Watch\nMade of black res...,stainless steel,simple,[{'product_asin': 'B000GB0G7A'}],True
5,What material is the bracelet made of in the C...,[Casio Men's Digital Watch with Stainless Stee...,The bracelet of the Casio Men's Digital Watch ...,multi_context,"[{'product_asin': 'B000GAYQJ0'}, {'product_asi...",True
6,What sets the Men's G-Shock DW5600E-1V Black R...,[Men's G-Shock DW5600E-1V Black Resin Quartz W...,The answer to given question is not present in...,multi_context,[{'product_asin': 'B000GAYQKY'}],True
7,How does the Men's Sergeants Verdict Fashion B...,[Men's Sergeants Verdict Fashion Boot \n\nMade...,The Men's Sergeants Verdict Fashion Boot has a...,multi_context,"[{'product_asin': 'B000G377WA'}, {'product_asi...",True
8,What timekeeping mechanism is used in the Casi...,[Casio Men's Digital Watch with Stainless Stee...,The Casio Men's Digital Watch with Stainless S...,multi_context,"[{'product_asin': 'B000GAYQJ0'}, {'product_asi...",True
9,What makes the Women's Star Ox M5039c' Sneaker...,[Women's Star Ox M5039c' Sneakers are low-top ...,The Women's Star Ox M5039c' Sneakers are uniqu...,reasoning,[{'product_asin': 'B0002M30XA'}],True


In [10]:
type(testset.to_dataset())

datasets.arrow_dataset.Dataset

In [80]:
questions = {
    "brand": "What brand does this product belong go? Return back only the name of the brand",
    "weather": "Which of this best suit the condition this product can be used in Summer, Winter or Transitional to indicate it could be used in between summer and winter",
    "usage": "Is this item best for Casual, Sport, Athleisure, Format, or Outdoor",
    "type": "Is this item an Outerwear, Footwear, Accessories, Tops, Trousers or Suits?",
    "material": "What material is used to make this product, is it Wool, Polyester, Leather? Return back only the material name",
    "color": "What is the primary most obvious colour of this item? Return back only the colour",
    "target_audience": "Who are the target aduience for this item Kids, Teens, Adults, Any, Kids and Teens, Kids and Adults or Teens and Adults?"
}


prompt = """Answer the following questions about this product and return back your answers with the questions' key while replacing the value with your answer for each question.



Questions:
{questions}

Product Details:
{product_details}

Product Image Description:
{image_description}

If you do not have an answer, simply return 'Others'
If this item is not a fashion item, return INVALID for all the answers.

Only respond with a correct JSON answer.
{{
    "brand": "",
    "weather": "",
    "usage": "",
    "type": "",
    "material": "",
    "color": "",
    "target_audience": ""
}}
"""


In [112]:
from langchain.prompts.chat import ChatPromptTemplate
from langchain_core.exceptions import OutputParserException
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from pydantic import BaseModel, Field, ValidationError
from datasets import Dataset 

class ProductAnalysisItem(BaseModel):
    name: str = Field(description="Name of the product")
    product_asin: str = Field(description="Identification number")
    total_customers_that_rated: str = Field(description="The number of people that rater this product")
    overall_ratings: str = Field(description="The ratings for this product")
    brand: str = Field(description="The brand name for this item")
    available_written_reviews: str = Field(description="The number of people that wrote review for this ietm")
    weather: str = Field(description="Best weather this item can be used in.")
    usage: str = Field("In what situation can this item be used for")
    type: str = Field(description="What is the best category this item can be placed in")
    material: str = Field(description="What is the product made of?")
    color: str = Field(description="What is the colour of the product?")
    target_audience: str = Field(description="Who is the best audience this item is targetted to")
    price: str = Field(description="The price tag for this product.")
    currency: str = Field(description="The currency for the product's price")
    ciam_category: str = Field(description="The CIAM category for this item")
    

model_name = TextLLM.LLAMA_3_1_INSTRUCT
model = ChatOllama(model=model_name)
chain_prompt = ChatPromptTemplate.from_template(prompt)

output = {}
analysis_items = []
try_times = 1
max_retries = 3

for index, product_doc in enumerate(product_info_docs[0:5]):
    prodct_combo_instance = products_combined_infos[index][1].product
    while try_times < max_retries:
        try:   
            image_description = product_image_descriptions[index]
            chain = chain_prompt | model | JsonOutputParser()
            output = chain.invoke({
                "questions": questions,
                "product_details": product_doc.page_content,
                "image_description": image_description,
            })

            output['price'] = str(prodct_combo_instance.price)
            output['product_asin'] = prodct_combo_instance.product_asin
            output['name'] = prodct_combo_instance.name
            output['overall_ratings'] = str(prodct_combo_instance.overall_ratings)
            output['available_written_reviews'] = str(len(prodct_combo_instance.reviews))
            output['total_customers_that_rated'] = str(prodct_combo_instance.total_customers_that_rated)
            output['currency'] = prodct_combo_instance.currency
            output['ciam_category'] = prodct_combo_instance.category
            _analysis_item = ProductAnalysisItem.model_validate(output)
            analysis_items.append(_analysis_item.model_dump())
            break
        except (ValidationError, OutputParserException) as e:
            try_times += 1
            
            if try_times == max_retries:
                print(f"\n\nSkipping after {max_retries} retries. {prodct_combo_instance.product_asin} due to ane error {e}\n")
                raise e
    
    index += 1

In [113]:
analysis_items

[{'name': "Official Blippi: Let's Look and Find!",
  'product_asin': '070231062X',
  'total_customers_that_rated': '325',
  'overall_ratings': '4.7',
  'brand': 'Blippi',
  'available_written_reviews': '0',
  'weather': 'Transitional',
  'usage': 'Casual',
  'type': 'INVALID',
  'material': 'Others',
  'color': 'Blue',
  'target_audience': 'Kids',
  'price': '5.99',
  'currency': '£',
  'ciam_category': 'Fashion'},
 {'name': 'F91W-1 Casual Sport Watch',
  'product_asin': 'B00006I551',
  'total_customers_that_rated': '5914',
  'overall_ratings': '4.6',
  'brand': 'CASIO',
  'available_written_reviews': '0',
  'weather': 'Transitional',
  'usage': 'Casual, Sport',
  'type': 'Accessories',
  'material': 'Others',
  'color': 'Black',
  'target_audience': 'Any',
  'price': '15.14',
  'currency': '£',
  'ciam_category': 'Fashion'},
 {'name': 'Chucks Red M9621C Red All Star HI',
  'product_asin': 'B000072UPN',
  'total_customers_that_rated': '7691',
  'overall_ratings': '4.7',
  'brand': 'Con